#### Initialize ####

In [ ]:
function tf {aws-vault exec terraform_user -- terraform $args}

function prompt_to_confirm { 
    $user_response = Read-host "Continue? y/n or 1/0"
    if ($user_response -ne "y" -and $user_response -ne 1) {break}
}

In [ ]:
$aws_region = "us-east-2"
$env:AWS_REGION=$aws_region #for aws_vault
$env:TF_VAR_aws_region=$aws_region #for terraform

In [ ]:
terraform init #-upgrade

#### Workflow ####

In [ ]:
terraform fmt

In [ ]:
terraform validate

In [ ]:
$keyname = "ssh_key"
$env:TF_VAR_ssh_keyname=$keyname
#ssh-keygen -t rsa -b 4096 -f $keyname -N "" -C $keyname.pem

In [ ]:
tf plan -out=tfplan
terraform show -json tfplan > plan.json

In [ ]:
#Output planned resource addresses and their names in the cloud

$plan = Get-Content -Raw -Path "plan.json" | ConvertFrom-Json

$resourcesToDeploy = $plan.resource_changes | Where-Object { 
    $_.change.actions -contains "create" -or $_.change.actions -contains "update" 
} | ForEach-Object { 
    "Type: " + $_.type
    "TF name: " + $_.name
    "Cloud name: " + $_.change.after.name
    "Name tag: " + $_.change.after.tags_all.name
    ""
}

$resourcesToDeploy
Get-Date

In [ ]:
###      ###
### PLAN ###
###      ###

tf plan
Get-Date

In [ ]:
###       ###
### APPLY ###
###       ###

#try {prompt_to_confirm} catch {break}
tf apply -auto-approve
Get-Date

In [ ]:
$public_ip = terraform output -raw public_ip
$http_port = terraform output -raw http_port

In [ ]:
(Invoke-WebRequest -Uri ("http://" + $public_ip + ":$http_port")).Content

In [ ]:
###         ###
### DESTROY ###
###         ###

try {prompt_to_confirm} catch {break}
tf destroy -auto-approve
Get-Date

#### Info ####

In [ ]:
terraform version

In [ ]:
#Root module configuration
cat .\main.tf

In [ ]:
($resources = terraform state list) | Out-Host

In [ ]:
foreach ($resource in $resources) { terraform state show $resource }

In [ ]:
terraform output

#### Remote access ####

In [ ]:
Test-NetConnection -ComputerName $public_ip -Port 22

In [ ]:
puttygen "$keyname.pem" -O private -o "$keyname.ppk"

In [ ]:
#Start SSH session

#Start ssh -A "ec2-user@$public_ip -i $keyname.pem"
Start "plink.exe" -A "-ssh -i $keyname.ppk ec2-user@$public_ip"

In [ ]:
#Execute single SSH command

#ssh ec2-user@$public_ip -i "$keyname.pem" 'cat /etc/xattr.conf'
plink -ssh -i "$keyname.ppk" "ec2-user@$public_ip" "cat /etc/xattr.conf"

#### Troubleshooting ####

In [ ]:
#$env:TF_LOG = "TRACE"
#$env:TF_LOG_PATH = "log.txt"
#$env:TF_LOG = ""

In [ ]:
tf apply -refresh-only -auto-approve